In [1]:
import json
import time
from tqdm import tqdm
from google import genai
from google.api_core.exceptions import ResourceExhausted

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [3]:
def save_progress(data, filename="gemini_responses_progress.json"):
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)
    print(f"Progress saved to {filename}")

In [4]:
def process_data(input_file="additions_error.json", output_file="gemini_responses_complete.json"):
    
    # Load the data
    with open(input_file, "r") as f:
        data = json.load(f)
    
    # Filter for entries where error is True
    error_entries = [entry for entry in data if entry.get("error") == True]
    print(f"Found {len(error_entries)} entries with errors to process")
    
    
    # Process each error entry
    for entry in tqdm(error_entries, desc="Processing error entries"):
        # Skip if already processed
        if "new_thought" in entry:
            continue
            
        # Construct the prompt
        prompt = f"Add {entry['numbers']}. Append to the thought process to correct the output: {entry['thought']}"
        
        try:
            # Make the API call
            response = client.models.generate_content(
                model="gemini-2.0-flash-thinking-exp",
                contents=prompt
            )
            
            # Store the response
            entry["new_thought"] = response.text
            
            # Small delay to avoid hitting rate limits too quickly
            time.sleep(10)
            
        except ResourceExhausted:
            print("\nRate limit reached! Saving progress...")
            save_progress(data, f"gemini_responses_partial_{int(time.time())}.json")
            print("Waiting 60 seconds before continuing...")
            time.sleep(60)
            
            # Try again after waiting
            try:
                response = client.models.generate_content(
                model="gemini-2.0-flash-thinking-exp",
                contents=prompt
                )
                entry["new_thought"] = response.text
            except Exception as e:
                print(f"Second attempt failed: {e}")
                entry["new_thought"] = "API ERROR: " + str(e)
                
        except Exception as e:
            print(f"\nError processing entry: {e}")
            entry["new_thought"] = "API ERROR: " + str(e)
    
    save_progress(data, output_file)
    print(f"Processing complete! Results saved to {output_file}")
    
    return data


In [5]:
result = process_data()

Found 1176 entries with errors to process


Processing error entries: 100%|██████████| 1176/1176 [6:52:50<00:00, 21.06s/it]  

Progress saved to gemini_responses_complete.json
Processing complete! Results saved to gemini_responses_complete.json


In [35]:
print(result)

[{'numbers': [8708368, 7348666, 1671933, 2269941, 3636139, 5857175], 'response': "<think>\nI need to add all the given numbers: 8,708,368; 7,348,666; 1,671,933; 2,269,941; 3,636,139; and 5,857,175.\n\nFirst, I'll list them all together: 8,708,368 + 7,348,666 + 1,671,933 + 2,269,941 + 3,636,139 + 5,857,175.\n\nI'll start by adding the first two numbers: 8,708,368 and 7,348,666. Adding them together gives me 16,057,034.\n\nNext, I'll add 1,671,933 to this sum. So, 16,057,034 + 1,671,933 equals 17,728,967.\n\nThen, I'll add the next number: 2,269,941. Adding that to 17,728,967 gives me 20,000,908.\n\nAfter that, I'll include 3,636,139 in my total. So, 20,000,908 + 3,636,139 equals 23,637,047.\n\nFinally, I'll add the last number: 5,857,175 to 23,637,047. This results in a final sum of 29,494,222.\n</think>\n\nSure! Let's calculate the sum step by step:\n\n\\[\n\\begin{align*}\n& 8,\\!708,\\!368 \\\\\n+ & 7,\\!348,\\!666 \\\\\n+ & \\underline{1,\\!671,\\!933} \\\\\n+ & 2,\\!269,\\!941 \\\\